In [2]:
## import 정보

from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import datetime
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
import time
import re


#### 셀레니움 안쓰고 오로지 bs, requests 를 사용해서
## 네이버가 bs 못쓰게 막아버림 ㅋ ajax와 쿠키의 미친콜라보

In [7]:
## BS 옵션

headers = {"User-Agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
                        AppleWebKit/537.36 (KHTML, like Gecko) \
                        Chrome/91.0.4472.101 Safari/537.36'}


## 기타 옵션

pd.set_option('display.unicode.east_asian_width', True)
re_title = re.compile('[^가-힣|a-z|A-Z ]')   # 정규 표현식

##사이트 옵션 & 카테고리 정보 $  DF 설정

url = ''
columns = [titles, writers, reviews, category1, category2, category3]



100번부터 340번까지 25가지 카테고리.
https://book.naver.com/category/index.naver?cate_code=350
뒤에 3자리씩 붙이는 방식으로 책 카테고리 세분화

https://book.naver.com/category/index.naver?cate_code=100010010&tab=top100&list_type=list&sort_type=publishday&page=1
한번 받아볼까

<a class="N=a:bta.title docs-creator" 
href="http://book.naver.com/bookdb/book_detail.naver?bid=13993822">알려지지 않은 예술가의 눈물과 자이툰 파스타</a>

파란색 글자 태그 따기


In [49]:
url= 'https://book.naver.com/category/index.naver?cate_code=100'
html= requests.get(url)
resp= BeautifulSoup(html.text, 'html.parser')
print(resp)

<!DOCTYPE HTML>

<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<title>
소설홈, 소설 : 네이버 책</title>
<meta content="article" property="og:type"/>
<meta content="네이버 책" property="og:title"/>
<meta content="http://book.naver.com/category/index.naver?cate_code=100" property="og:url"/>
<meta content="https://ssl.pstatic.net/static/m/book/icons/book_og_270x270.png" property="og:image"/>
<meta content="책으로 만나는 새로운 세상" property="og:description"/>
<meta content="" property="og:article:thumbnailUrl"/>
<meta content="네이버 책" property="og:article:author"/>
<meta content="https://book.naver.com" property="og:article:author:url"/>
<link href="https://www.naver.com/favicon.ico?book" rel="shortcut icon" type="image/x-icon"/>
<script type="text/javascript">
var __bLogin = false;
var __bAdult = false;
</script>
<link href="/css/release/base_20211207091736.css" rel="stylesheet" type="text/css"/>
<script charse

resp 확인. columns = [titles, writers, url_list] 

In [9]:
url_list = []
temp = ''

for i in range(1,6):
    url = 'https://book.naver.com/category/index.naver?cate_code=100010010&tab=top100&list_type=list&sort_type=publishday&page={}'.format(i)
    html = requests.get(url, headers = headers)
    resp = BeautifulSoup(html.text , 'html.parser')
    resp.findAll('',{'class' : "N=a:bta.title docs-creator" })
    temp = resp.attrs['href']
    url_list.append(temp)
print(url_list)

KeyError: 'href'

In [21]:
url = 'https://book.naver.com/category/index.naver?cate_code=100010010&tab=top100&list_type=list&sort_type=publishday&page={}'.format(i)
html = requests.get(url, headers = headers)
resp = BeautifulSoup(html.text , 'html.parser')
resp.findAll('a')


[<a href="#" name="start">본문시작</a>,
 <a href="/index.naver" id="home_navi">책 홈</a>,
 <a href="#" id="category_location1_depth" onclick="showLocationLayer('location1_depth');return false;" onmouseout="hideLocationLayer('location1_depth')">소설</a>,
 <a href="#" id="category_location2_depth" onclick="showLocationLayer('location2_depth');return false;" onmouseout="hideLocationLayer('location2_depth')">나라별 소설</a>,
 <a href="#" id="category_location3_depth" onclick="showLocationLayer('location3_depth');return false;" onmouseout="hideLocationLayer('location3_depth')">한국소설</a>,
 <a class="N=a:bil.category1,i:100" href="http://book.naver.com/category/index.naver?cate_code=100">소설</a>,
 <a class="N=a:bil.category1,i:110" href="http://book.naver.com/category/index.naver?cate_code=110">시/에세이</a>,
 <a class="N=a:bil.category1,i:160" href="http://book.naver.com/category/index.naver?cate_code=160">경제/경영</a>,
 <a class="N=a:bil.category1,i:170" href="http://book.naver.com/category/index.naver?cate_code

In [34]:
url = 'https://book.naver.com/category/index.naver?cate_code=100010010&tab=top100&list_type=list&sort_type=publishday&page={}'.format(i)
html = requests.get(url, headers = headers)
resp = BeautifulSoup(html.text , 'html.parser')
s = resp.findAll('a', class_ = "N=a:bta.title")
for i in s:
    url_list.append(i.attrs['href'])
    

## 12개밖에 안쏴주네?

In [35]:
url_list

['http://book.naver.com/bookdb/book_detail.naver?bid=13314689',
 'http://book.naver.com/bookdb/book_detail.naver?bid=15512482',
 'http://book.naver.com/bookdb/book_detail.naver?bid=15476484',
 'http://book.naver.com/bookdb/book_detail.naver?bid=15475909',
 'http://book.naver.com/bookdb/book_detail.naver?bid=15475641',
 'http://book.naver.com/bookdb/book_detail.naver?bid=15509935',
 'http://book.naver.com/bookdb/book_detail.naver?bid=15432263',
 'http://book.naver.com/bookdb/book_detail.naver?bid=15369087',
 'http://book.naver.com/bookdb/book_detail.naver?bid=15372732',
 'http://book.naver.com/bookdb/book_detail.naver?bid=15372716',
 'http://book.naver.com/bookdb/book_detail.naver?bid=15372717',
 'http://book.naver.com/bookdb/book_detail.naver?bid=15133927']

In [40]:
url = 'https://book.naver.com/category/index.naver?cate_code=100010010&tab=top100&list_type=list&sort_type=publishday&page=1'
html = requests.get(url, headers = headers)
resp = BeautifulSoup(html.text , 'html.parser')
resp

## bs 로는 막힘 아약스가 막아버림

<script src="/js/jquery-1.9.1.min.js?20211207091736" type="text/javascript"></script>
<script src="/js/logeye.js" type="text/javascript"></script>
<script type="text/javascript">
var ccsrv = "cc.naver.com";
var nclk_evt = 3;

nclk_do();

function emptyImg(obj) {
	obj.src = "https://ssl.pstatic.net/static/book/image/noimg4.gif";
	obj.width = "106";
	obj.height = "154";
}

function callEyelogApi(storeId) {
	if(!storeId) {
		return ;
	}

	var obj = {
		'storeId':storeId
	}

	logeye.call.setParams(obj);
	logeye.call.callLogEye();
}
</script>
<div id="content">
<div class="blind" id="start"><a href="#" name="start">본문시작</a></div>
<div class="location">
<h2 class="blind">한국소설</h2>
<ul class="navi">
<li class="home"><a href="/index.naver" id="home_navi">책 홈</a></li>
<li>&gt;</li>
<li class="select">
<a href="#" id="category_location1_depth" onclick="showLocationLayer('location1_depth');return false;" onmouseout="hideLocationLayer('location1_depth')">소설</a>
</li>
<li>&gt;</li>
<li class="selec

# 여기서부터는 셀레니움 코드

/html/body/div/div[3]/div[2]/div[9]/ol/li[1]/dl/dt/a
/html/body/div/div[3]/div[2]/div[9]/ol/li[2]/dl/dt/a
/html/body/div/div[3]/div[2]/div[9]/ol/li[20]/dl/dt/a

In [3]:
# 셀레니움 옵션.

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
options.add_argument('lang=ko_KR')
options.add_argument('disable_gpu')
driver = webdriver.Chrome('./chromedriver', options=options)

C:\Users\Nokdu\.conda\envs\ai_exam\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  if __name__ == '__main__':


In [50]:
temp = None
url_list=[]

In [ ]:
for x in range(10,34):
    for y in range(1,20):
        try:
            for i in range(1,6):
                url = 'https://book.naver.com/category/index.naver?cate_code={}0{}0&tab=top100&list_type=list&sort_type=publishday&page={}'.format(x,y,i)
                driver.get(url)
                for j in range(1,21):
                    temp = driver.find_element_by_xpath('/html/body/div/div[3]/div[2]/div[9]/ol/li[{}]/dl/dt/a'.format(j))
                    new_url = temp.get_attribute("href")
                    url_list.append(new_url)
        except:
            print()
            print(url_list)

In [ ]:
for x in range(10, 12):
    #10부터 34까지 있고 중간에 없는것도 있음
    for y in range(1, 20):

        ## 100개 마다
        for i in range(1,6):
            try:
                url = 'https://book.naver.com/category/index.naver?cate_code={}0{}0&tab=top100&list_type=list&sort_type=publishday&page={}'.format(x,y,i)
                driver.get(url)
                for j in range(1,21):
                    temp = driver.find_element_by_xpath('/html/body/div/div[3]/div[2]/div[9]/ol/li[{}]/dl/dt/a'.format(j))
                    new_url = temp.get_attribute("href")
                    url_list.append(new_url)
            except:
                print('.', end='')
                # print(len(url_list))
    try:
        for z in range(100):
            driver.get(url_list[z])
            titles = []
            writers = []
            reviews = []
            title = driver.find_element_by_xpath('/html/body/div[1]/div[3]/div[4]/div[1]/h2/a').text
            writer = driver.find_element_by_xpath('//*[@id="container"]/div[4]/div[1]/div[2]/div[2]/a[1]').text
            driver.find_element_by_xpath('/html/body/div[1]/div[3]/div[5]/ul/li[3]/a').click()
            #리뷰 페이지 클릭
            #1개의 리뷰
            try:
                for a in range(2,4):
                    ## 리뷰 30개만.
                    for b in range(1, 11):
                        driver.find_element_by_xpath('/html/body/div/div[3]/div[6]/div/ul/li[{}]/dl/dt/a'.format(b)).click
                        time.sleep(0.5)
                        review = driver.find_element_by_xpath('/html/body/div[6]/div[1]/div[2]/div[2]/div[2]/div[2]/div/div/div[8]/div[1]/div/table[2]/tbody/tr/td[2]/div[1]/div/div/div[3]').text
                        titles.append(title)
                        reviews.append(review)
                        writers.append(writer)
                        driver.back()
            except:
                print('.')
        df_review_100 = pd.DataFrame({'titles': titles, 'writers': writers, 'reviews':reviews})
        df_review_100.to_csv('./datasets/review_{}_{}_cate'.format(x,y))
    except:
        print('이 카테고리는 없습니다.')

